In [1]:
import pandas as pd
import numpy as np
import os
import datetime
from matplotlib import pyplot as plt
import seaborn as sns
import scipy.stats as stats
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')
from IPython.display import FileLink

import itertools

from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import OneHotEncoder
from scipy.sparse import hstack


pd.set_option('display.max_columns', 100)

PATH_TO_DATA = 'data/'
train = pd.read_csv(os.path.join(PATH_TO_DATA, 'train_new.csv'))
test = pd.read_csv(os.path.join(PATH_TO_DATA, 'test_new.csv'))

Датасеты train и test уже предобработанные. (New_features.ipynb)

In [2]:
train.head()

,orderid,userid,delta_buy_buy,full_price_scaled,buy_month,fly_month,buy_number,reg_ind,field6,field7,field8,number_tickets_children_1,field10,fly_hour,field12,field13,flight_duration_scaled,number_tickets,delta_buy_fly,field17,buy_weekday,field19,fly_weekday,buy_year,field22,field23,number_tickets_adult,field25,field26,field27,number_tickets_children_2,buy_quarter,indicator_goal21,indicator_goal22,indicator_goal23,indicator_goal24,indicator_goal25,goal21,goal22,goal23,goal24,goal25,goal1,number_all_buy,ticket_price,full_price_int,flight_duration_int,cumsum_delta_buy_buy,delta_fly,quantity_buy_one_day,...,mean_field13_user,std_field13_user,mean_field17_user,std_field17_user,mean_field19_user,std_field19_user,mean_field22_user,std_field22_user,mean_field25_user,std_field25_user,mean_field26_user,std_field26_user,mean_field27_user,std_field27_user,mean_field6_user,std_field6_user,mean_field7_user,std_field7_user,mean_field8_user,std_field8_user,mean_delta_fly_user,std_delta_fly_user,mean_quantity_buy_one_day_user,std_quantity_buy_one_day_user,mean_quantity_buy_one_day_averaged_user,std_quantity_buy_one_day_averaged_user,mean_mean_quantity_buy_one_day_user,std_mean_quantity_buy_one_day_user,mean_delta_fly_fly_user,std_delta_fly_fly_user,mean_ticket_price_user,std_ticket_price_user,mean_ratio_delta_bf_ff_user,std_ratio_delta_bf_ff_user,mean_ratio_delta_qbod_bf_user,std_ratio_delta_qbod_bf_user,mean_ratio_price_duration_user,std_ratio_price_duration_user,mean_ratio_13_17_user,std_ratio_13_17_user,mean_ratio_22_25_user,std_ratio_22_25_user,mean_ratio_26_27_user,std_ratio_26_27_user,mean_ratio_13_22_user,std_ratio_13_22_user,mean_ratio_17_25_user,std_ratio_17_25_user,mean_ratio_6_12_user,std_ratio_6_12_user
0,0,10d654494cbe97bbb25d51ead2600679aff9e097924add...,-999,-0.626508,11,12,1,1,0,1,1,0,0,9,5,20,-0.661308,1,-999,1,4,1,1,1,1,15,1,1,2,1,0,4,1,1,0,1,1,0,1,0,0,0,0,1,-0.626508,-8,-5,0,4,1,...,20.0,0.000000,1.0,0.0,1.0,0.0,1.000000,0.000000,1.000000,0.000000,2.000000,0.000000,1.000000,0.000000,0.0,0.000000,1.000000,0.000000,1.0,0.0,4.000000,0.000000,1.0,0.0,1.0,0.0,1.0,0.0,0.000000,0.000000,-0.626508,0.000000,1.609438,0.000000,-0.916291,0.000000,0.097806,0.000000,2.351375,0.000000,0.000000,0.000000,0.405465,0.000000,2.351375,0.000000,0.000000,0.000000,-1.791759,0.000000
1,1,4aafc0391f72bbcf60537aece62923baf9ce644b64ac36...,144,-0.393794,5,7,2,0,0,2,1,0,0,21,125,3,-0.101043,1,57,1,5,1,6,2,121,15,1,41,3,1,0,2,1,1,0,1,0,0,0,0,0,0,0,2,-0.393794,-5,-1,144,201,1,...,23.5,20.500000,1.0,0.0,1.5,0.5,102.500000,18.500000,35.500000,5.500000,7.500000,4.500000,2.000000,1.000000,0.0,0.000000,2.000000,0.000000,1.0,0.0,120.500000,80.500000,1.0,0.0,1.0,0.0,1.0,0.0,80.500000,80.500000,-0.393794,0.000000,1.343209,2.370363,-3.193860,0.173435,-0.309327,0.084689,1.903331,1.210184,1.037508,0.028844,0.935901,0.242754,-2.026858,1.390869,-2.892681,0.151841,-4.181321,0.654961
2,2,bac8ffef46348f587c8d17137ab01fb24aef21547c647d...,134,-0.548937,2,3,2,0,0,1,1,0,1,7,4,3,-0.661308,1,4,1,2,3,6,2,17,14,1,1,11,7,0,1,1,1,0,1,1,0,0,0,0,0,0,2,-0.548937,-7,-5,134,138,1,...,4.0,1.000000,1.0,0.0,2.0,1.0,9.000000,8.000000,1.000000,0.000000,11.000000,0.000000,7.000000,0.000000,0.0,0.000000,1.000000,0.000000,1.0,0.0,70.500000,67.500000,1.0,0.0,1.0,0.0,1.0,0.0,67.500000,67.500000,-0.587722,0.038786,-0.958461,2.344756,-0.804719,0.111572,0.192162,0.094356,0.895880,0.202733,1.098612,1.098612,0.405465,0.000000,-0.202733,1.301345,0.000000,0.000000,-2.158744,0.549306
3,3,0392247b4b87674aba2c32bf2292b105771a6a376871be...,-999,-0.238651,10,11,1,1,3,2,1,0,0,19,12,3,-0.521242,2,-999,1,3,3,4,1,46,12,2,18,1,1,0,4,1,1,0,1,1,0,0,0,0,0,0,3,-0.119326,-3,-4,0,22,1,...,23.0,28.284271,1.0,0.0,3.0,0.0,32.666667,22.484563,6.666667,8.013877,2.333333,1.885618,1.666667,0.942809,1.0,1.414214,1.333333,0.471405,1.0,0.0,219.333333,140.072204,1.0,0.0,1.0,0.0,1.0,0.0,103.666667,125.990299,-0.328162,0.151027,0.255895,2.340512,-2.588046,0.272914,0.095257,0.266594,1.617343,1.307011,1.387935,1.373124,0.135155,0.

Добавим категориальные признаки

In [3]:
def new_cat_features(df):
    list_to_str = [
        'buy_month',
        'fly_month',
        'flight_duration_int',
        'full_price_int',
        'number_tickets',
        'quantity_buy_one_day',
        'fly_weekday',
        'fly_hour',
        'buy_weekday',
        'buy_year',
        'field10',
        'field7',
        'field8',
        'field27'
    ]
        
    for column in list_to_str:
        df[column + '_str'] = df[column].astype(str)
        
new_cat_features(train)
new_cat_features(test)

In [4]:
drop_columns = ['goal21', 'goal22', 'goal23', 'goal24', 'goal25', 'goal1', 'userid', 'orderid']
X = train.drop(columns=drop_columns)
X_test = test.drop(columns=['userid', 'orderid'])
y = train['goal1']

In [5]:
n_fold = 5
folds = StratifiedKFold(n_splits=n_fold, shuffle=True, random_state=42)

train_model ищет для каждого фолда лучшую модель и на ней делает предсказание. Все эти предсказания суммируются и усредняются по всем фолдам.  
Также внутри для категориальных фичей применяется OHE.

In [6]:
import time
import lightgbm as lgb

def train_model(X, X_test, y, params, folds):
    oof = np.zeros(len(X))
    prediction = np.zeros(len(X_test))
    scores = []
    feature_importance = pd.DataFrame()

    cat_features = X.columns[X.dtypes == 'object']
    notcat_features = X.columns[X.dtypes != 'object']
    enc = OneHotEncoder(handle_unknown='ignore')
    full = pd.concat([X, X_test])
    enc_ohe = enc.fit(full[cat_features])

    X_test_ohe = enc.transform(X_test[cat_features])
    X_test_sparse = hstack([X_test.drop(columns=cat_features), X_test_ohe])
        
    
    for fold_n, (train_index, valid_index) in enumerate(folds.split(X, y)):
        print('Fold', fold_n, 'started at', time.ctime())
        X_train, X_valid = X.loc[train_index], X.loc[valid_index]
        y_train, y_valid = y[train_index], y[valid_index]
    
        X_train_ohe = enc.transform(X_train[cat_features])
        X_train_sparse = hstack([X_train.drop(columns=cat_features), X_train_ohe])
        X_valid_ohe = enc.transform(X_valid[cat_features])

        X_valid_sparse = hstack([X_valid.drop(columns=cat_features), X_valid_ohe])
        

        train_data = lgb.Dataset(X_train_sparse, label=y_train)
        valid_data = lgb.Dataset(X_valid_sparse, label=y_valid)
        
        model = lgb.train(params,
                train_data,
                num_boost_round=20000,
                valid_sets = [train_data, valid_data],
                verbose_eval=1000,
                early_stopping_rounds = 800)

        y_pred_valid = model.predict(X_valid_sparse)
        y_pred = model.predict(X_test_sparse, num_iteration=model.best_iteration)

        oof[valid_index] = y_pred_valid.reshape(-1,)
        scores.append(roc_auc_score(y_valid, y_pred_valid))
        
        prediction += y_pred

    prediction /= n_fold
    print('CV mean score: {0:.4f}, std: {1:.4f}.'.format(np.mean(scores), np.std(scores)))
    
    return oof, prediction, scores

In [7]:
%%time

params = {
    'boost': 'gbdt',
    'feature_fraction': 0.1,
    'learning_rate': 0.01, 
    'max_depth': 7, 
    'metric': 'auc',
    'min_data_in_leaf': 150,
    'max_bin': 64,
    'num_leaves': 50, 
    'num_threads': -1,
    'verbosity': 1,
    'objective': 'binary',
    'seed': 42
}

oof_lgb, prediction_lgb, scores = train_model(X, X_test, y, params=params, folds=folds)

Fold 0 started at Fri Jan 24 19:48:54 2020
Training until validation scores don't improve for 800 rounds.
[1000]	training's auc: 0.872972	valid_1's auc: 0.698464
Early stopping, best iteration is:
[1090]	training's auc: 0.879075	valid_1's auc: 0.69883
Fold 1 started at Fri Jan 24 19:49:58 2020
Training until validation scores don't improve for 800 rounds.
[1000]	training's auc: 0.871219	valid_1's auc: 0.718551
Early stopping, best iteration is:
[1081]	training's auc: 0.877051	valid_1's auc: 0.718832
Fold 2 started at Fri Jan 24 19:50:58 2020
Training until validation scores don't improve for 800 rounds.
[1000]	training's auc: 0.874559	valid_1's auc: 0.718154
Early stopping, best iteration is:
[1004]	training's auc: 0.874879	valid_1's auc: 0.718213
Fold 3 started at Fri Jan 24 19:51:56 2020
Training until validation scores don't improve for 800 rounds.
[1000]	training's auc: 0.869564	valid_1's auc: 0.727488
[2000]	training's auc: 0.916935	valid_1's auc: 0.728066
Early stopping, best ite

### CV mean score: 0.7169, std: 0.0098.
### pub: 0.693331  
### priv: 0.693581